<a href="https://colab.research.google.com/github/affanahmed373/Machinelearn-deeplearn-projects/blob/main/car_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
IMG_SIZE = 224

Efficientnet baseline 0 version 1 is used here for image classification

The Classification uses car dataset from Stanford which is available in Tensorflow datasets

In [2]:


import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras.applications import EfficientNetB0

In [3]:
model = EfficientNetB0

In [4]:
import tensorflow_datasets as tfds

batch_size = 64

dataset_name = "cars196"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/8144 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cars196/2.1.0.incompleteW0WRDW/cars196-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/8041 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cars196/2.1.0.incompleteW0WRDW/cars196-test.tfrecord*...:   0%|          |…

Dataset cars196 downloaded and prepared to /root/tensorflow_datasets/cars196/2.1.0. Subsequent calls will reuse this data.


In [5]:
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

In [6]:
# One-hot / categorical encoding
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)


Transfer Learning and Evaluation metrics are included

In [11]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = inputs
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy",
         metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
    )
    return model


In [12]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = build_model(num_classes=NUM_CLASSES)

epochs = 25  # @param {type: "slider", min:8, max:80}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/25
127/127 - 68s - loss: 4.5180 - accuracy: 0.2453 - precision: 0.3544 - recall: 0.1948 - val_loss: 2.7748 - val_accuracy: 0.3455 - val_precision: 0.8070 - val_recall: 0.0601 - 68s/epoch - 538ms/step
Epoch 2/25
127/127 - 51s - loss: 1.2578 - accuracy: 0.6661 - precision: 0.7862 - recall: 0.5869 - val_loss: 2.1336 - val_accuracy: 0.4586 - val_precision: 0.7141 - val_recall: 0.2676 - 51s/epoch - 401ms/step
Epoch 3/25
127/127 - 52s - loss: 0.7990 - accuracy: 0.7801 - precision: 0.8523 - recall: 0.7186 - val_loss: 2.0993 - val_accuracy: 0.5055 - val_precision: 0.6485 - val_recall: 0.4056 - 52s/epoch - 407ms/step
Epoch 4/25
127/127 - 57s - loss: 0.6194 - accuracy: 0.8227 - precision: 0.8770 - recall: 0.7819 - val_loss: 2.4414 - val_accuracy: 0.5085 - val_precision: 0.5990 - val_recall: 0.4530 - 57s/epoch - 446ms/step
Epoch 5/25
127/127 - 60s - loss: 0.5691 - accuracy: 0.8410 - precision: 0.8822 - recall: 0.8082 - val_loss: 2.6936 - val_accuracy: 0.5089 - val_precision: 0.5811 - val_

In [13]:

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", 
                  metrics=["accuracy"])


unfreeze_model(model)

epochs = 10  # @param {type: "slider", min:8, max:50}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/10
127/127 - 61s - loss: 0.2453 - accuracy: 0.9413 - val_loss: 3.9330 - val_accuracy: 0.5440 - 61s/epoch - 481ms/step
Epoch 2/10
127/127 - 50s - loss: 0.1537 - accuracy: 0.9563 - val_loss: 3.9000 - val_accuracy: 0.5642 - 50s/epoch - 396ms/step
Epoch 3/10
127/127 - 50s - loss: 0.1317 - accuracy: 0.9646 - val_loss: 3.8724 - val_accuracy: 0.5669 - 50s/epoch - 395ms/step
Epoch 4/10
127/127 - 51s - loss: 0.1208 - accuracy: 0.9694 - val_loss: 3.9058 - val_accuracy: 0.5732 - 51s/epoch - 404ms/step
Epoch 5/10
127/127 - 51s - loss: 0.0982 - accuracy: 0.9727 - val_loss: 3.9089 - val_accuracy: 0.5809 - 51s/epoch - 401ms/step
Epoch 6/10
127/127 - 52s - loss: 0.0840 - accuracy: 0.9764 - val_loss: 3.9404 - val_accuracy: 0.5791 - 52s/epoch - 408ms/step
Epoch 7/10
127/127 - 51s - loss: 0.0813 - accuracy: 0.9765 - val_loss: 3.6871 - val_accuracy: 0.5786 - 51s/epoch - 401ms/step
Epoch 8/10
127/127 - 51s - loss: 0.0658 - accuracy: 0.9803 - val_loss: 3.7322 - val_accuracy: 0.5822 - 51s/epoch - 398

- Due to tensorflow mirrored strategy library issue I cannot use precison and recall again when unfreezing weights for transfer learning 
-parts of the solution are authored by tensorflow keras example